In [ ]:
## TODO UPDATE THE ID BEFORE RUNING THE NOTEBOOK
## If you are unsure how to collect the different ID, please ask your instructor

# Your IBM Cloud API Key, required to authenticate yourself when accessing the services using the python SDK
CLOUD_API_KEY = "*************************"

# Your ModelManagement/OpenScale Instance ID that has been provisionned for you
OPENSCALE_INSTANCE_ID = "*************************" # Update this to refer to a particular service instance

# The watsonx.ai production deployement space id where the Agentic RAG Langchain  model is deployed
SPACE_ID = "*************************"

# The unique name of the watsonx.ai Agentic RAG Langchain Developement like "Agentic RAG using LangChain"
AGENTIC_RAG_LC_DEV_NAME = "*************************"

# The prompt template asset id created in the watsonx.ai space for the Use Case
SPACE_PROMPT_TEMPLATE_ID = "*************************"

In [ ]:
!pip install -U ibm-watson-openscale | tail -n 1

In [ ]:
IAM_URL = "https://iam.cloud.ibm.com"
FACTSHEET_URL = "https://eu-de.dataplatform.cloud.ibm.com"
SERVICE_URL = "https://eu-de.aiopenscale.cloud.ibm.com"

credentials = {
    "url": "https://eu-de.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY,
}
CREDENTIALS = credentials

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

SERVICE_URL="https://eu-de.aiopenscale.cloud.ibm.com"

authenticator = IAMAuthenticator(
    apikey=CLOUD_API_KEY,
    url=IAM_URL
)
wos_client = APIClient(
    authenticator=authenticator,
    service_url=SERVICE_URL,
    service_instance_id=OPENSCALE_INSTANCE_ID
)
data_mart_id = wos_client.service_instance_id
print(wos_client.version)

In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

gen_ai_evaluator = wos_client.integrated_systems.add(
    name="RAG Metrics Evaluator",
    description="RAG Metrics Evaluator",
    type="generative_ai_evaluator",
    parameters={"evaluator_type": "watsonx.ai", "model_id": "meta-llama/llama-3-3-70b-instruct"},
    credentials={
        "url": "https://eu-de.ml.cloud.ibm.com",
        "wml_location": "cloud",
        "apikey": CLOUD_API_KEY,
    },
)

# get evaluator integrated system ID
result = gen_ai_evaluator.result._to_dict()
evaluator_id = result["metadata"]["id"]
evaluator_id

In [ ]:
label_column = "reference"
context_fields = ["context"]
question_field = "query"
operational_space_id = "production"
problem_type= "retrieval_augmented_generation"
input_data_type= "unstructured_text"

monitors = {
    "generative_ai_quality": {
        "parameters": {
            "generative_ai_evaluator": { # global LLM as judge configuration
               "enabled": True,
               "evaluator_id": evaluator_id,
            },
            "min_sample_size": 2,
            "metrics_configuration": {
                "faithfulness": {
                    # "metric_prompt_template": "", # adding custom template
                    # Uncomment generative_ai_evaluator to use a different evaluator for this metric.
                    # Takes higher precedence than the generative_ai_evaluator specified at global level.
                    # "generative_ai_evaluator": {  # metric specific LLM as judge configuration
                    #     "enabled": True,
                    #     "evaluator_id": evaluator_id,
                    # },
                },
                "answer_relevance": {
                    # "metric_prompt_template": "", # adding custom template
                    # Uncomment generative_ai_evaluator to use a different evaluator for this metric
                    # Takes higher precedence than the generative_ai_evaluator specified at global level.
                    # "generative_ai_evaluator": {  # metric specific LLM as judge configuration
                    #     "enabled": True,
                    #     "evaluator_id": evaluator_id,
                    # },
                },
                "rouge_score": {},
                "exact_match": {},
                "bleu": {},
                "unsuccessful_requests": {
                    # "unsuccessful_phrases": []
                },
                "hap_input_score": {},
                "hap_score": {},
                "pii": {},
                "pii_input": {},
                "retrieval_quality": {
                    # Uncomment generative_ai_evaluator to use a different evaluator for this metric
                    # Takes higher precedence than the generative_ai_evaluator specified at global level.
                    # "generative_ai_evaluator": {  # metric specific LLM as judge configuration
                    #     "enabled": True,
                    #     "evaluator_id": evaluator_id,
                    # },
                    # The metrics computed for retrieval quality are context_relevance, retrieval_precision, average_precision, reciprocal_rank, hit_rate, normalized_discounted_cumulative_gain
                    # "context_relevance": {
                    #     "metric_prompt_template": "", # adding custom template
                    # }
                },
                # Answer similarity metric is supported only when LLM as judge is configured. Uncomment only when using LLM as judge.
                "answer_similarity": {
                    # "metric_prompt_template": "", # adding custom template
                    # Uncomment generative_ai_evaluator to use a different evaluator for this metric
                    # Takes higher precedence than the generative_ai_evaluator specified at global level.
                    # "generative_ai_evaluator": {  # metric specific LLM as judge configuration
                    #     "enabled": True,
                    #     "evaluator_id": evaluator_id,
                    # },
                },
            },
        }
    }
}

response = wos_client.monitor_instances.mrm.execute_prompt_setup(
    prompt_template_asset_id=SPACE_PROMPT_TEMPLATE_ID, 
    space_id=SPACE_ID,
    label_column=label_column,
    context_fields = context_fields,     
    question_field = question_field,     
    operational_space_id=operational_space_id, 
    problem_type=problem_type,
    input_data_type=input_data_type, 
    supporting_monitors=monitors, 
    background_mode=False
)

result = response.result
result.to_dict()